In [2]:
firstrun = True

if firstrun != False:
  !pip install fredapi
  !pip install matplotlib
  !pip install plotly
  !pip install keras

  from fredapi import Fred
  fred = Fred(api_key='63769be4fe08f91aef948b301151587d')
  import yfinance as yf

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
!pip install keras-tuner --upgrade
import keras_tuner
from tensorflow.keras.callbacks import EarlyStopping
import time
from matplotlib.ticker import MaxNLocator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 10.9 MB/s eta 0:00:00
Using TensorFlow backend


In [3]:
def getData(ticker_symbol,startday,endday):
  we_stock_data = yf.download(ticker_symbol, start=startday, end=endday)
  we_daily_returns = we_stock_data['Adj Close'].pct_change() * 100
  we_stock_data['Daily_Return'] = we_stock_data['Adj Close'].pct_change() * 100

  CPI = fred.get_series('MEDCPIM158SFRBCLE',observation_start=startday)
  M30US = fred.get_series('MORTGAGE30US',observation_start=startday)
  UNRA = fred.get_series('UNRATE',observation_start=startday)
  SP500 = fred.get_series('SP500',observation_start=startday)
  NEWH = fred.get_series('HOUST',observation_start=startday)
  NASDAQ = fred.get_series('NASDAQCOM',observation_start=startday)
  CARSA = fred.get_series('TOTALSA',observation_start=startday)

  df = {}
  df = pd.concat([SP500, CPI, M30US, UNRA, NEWH, NASDAQ, CARSA], axis=1)
  data_names = ['SP500', 'CPI', 'M30US', 'UNRA', 'NEWH', 'NASDAQ', 'CARSA']
  df.columns = data_names

  df = df.ffill()
  df = df.fillna(method = 'ffill')
  df = df.fillna(method = 'backfill')
  we_stock_data = we_stock_data.ffill()
  we_stock_data = we_stock_data.fillna(method = 'ffill')
  we_stock_data = we_stock_data.fillna(method = 'backfill')
  we = pd.concat((we_stock_data, df),axis=1)
  we.dropna(inplace=True)
  we.to_csv('data.csv', index=False)

  return we

In [4]:
dataog = getData('BAC','2022-01-01','2023-11-05')

[*********************100%%**********************]  1 of 1 completed


In [32]:
import pandas as pd
import numpy as np
#data = pd.read_csv('wedata22-23.csv')
data = dataog
seq_length = 3
target_coln = 'Close'
dayspan = 10

target_col = data.columns.get_loc(target_coln)
date = data.index
print(date[-5:])

'''
data = [[t1,x1],[t2,x2],[t3,x3]]
'''

scalerinput = MinMaxScaler()
scaleroutput = MinMaxScaler()
orig = data[target_coln]
scaleroutput.fit(data[target_coln].values.reshape(-1,1))
data = scalerinput.fit_transform(data)
df = pd.DataFrame()
df[0] = [1,2,3,4,5,6,7]
df[1] = [11,22,33,44,55,66,77]
print(data.shape)
def create_sequences(data, seq_length, target_col, nextdays,date):
    datepred = date[-nextdays:]
    date = date[seq_length-1:-nextdays]
    X, y, xpred = [], [], []
    data = pd.DataFrame(data)  # Convert the data to a DataFrame for column referencing
    for i in range(len(data) - seq_length - nextdays + 1):
        X.append(data.iloc[i:i+seq_length, :].values)  # Use all columns as input
        y_sequence = [data.iloc[i+seq_length+j, target_col] for j in range(nextdays)]
        y.append(y_sequence)
    for i in range(nextdays):
        xpred.append(data.iloc[i-nextdays:i+seq_length, :].values)  # Use all columns as input

    return np.array(X), np.array(y), date, datepred, np.array(xpred)

nextdays = 5
x , y , date, datepred, xpred = create_sequences(data,seq_length,target_col,nextdays,date)
print(x.shape)
print(y.shape)

###para regresar inverse escalado####
#y = scaleroutput.inverse_transform(y)
#for i in range(x.shape[1]):
 # x[:,i,:] = scalerinput.inverse_transform(x[:,i,:])
##for i in range(x.shape[0]):
 #print(date[i],x[i,:,target_col], y[i], orig[i+seq_length])
for i in range(nextdays):
 print(datepred[i],xpred[i,:,target_col])

testsize = int(np.round(x.shape[0]*.1))
trainsize = int(np.round((x.shape[0]-testsize)*.9))
x_train = x[:trainsize,:,:]
y_train = y[:trainsize,:]
print(x_train.shape)
print(y_train.shape)
x_val = x[trainsize:-testsize,:,:]
y_val = y[trainsize:-testsize,:]
print(x_val.shape)
print(y_val.shape)
x_test = x[-testsize:,:,:]
y_test = y[-testsize:,:]
print(x_test.shape)
print(y_test.shape)


datetrain = date[:trainsize]
dateval = date[trainsize:-testsize]
datetest = date[-testsize:]
print(datetrain.shape)
print(dateval.shape)
print(datetest.shape)
print(date.shape)
print(datetrain.shape[0]+dateval.shape[0]+datetest.shape[0])

plt.plot(datetrain,x_train[:,-1,target_col])
plt.plot(dateval,x_val[:,-1,target_col])
plt.plot(datetest,x_test[:,-1,target_col])
plt.figure()
plt.plot(date[:],x[:,-1,target_col])

TimeSteps=x_train.shape[1]
TotalFeatures=x_train.shape[2]

'''
for i in range(x.shape[0]):
  print(x[i,:,target_col], y[i])

'''

'''
train = data
test = train.tail(dayspan+seq_length+1)
test = test.reset_index(drop=True)
train = train.iloc[:-dayspan]
train.reset_index(drop=True, inplace=True)
val = train.tail(valsize+seq_length+1)
val = val.reset_index(drop=True)
train = train.iloc[:-valsize]
train.reset_index(drop=True, inplace=True)

date = data.index
datetest = date[-(dayspan+seq_length+1):]
dateval = val.index
datetrain = date[:-dayspan]
dateval = datetrain[-(valsize+seq_length+1):]
datetrain = datetrain[:-valsize]

plt.plot(datetrain[-10:],train[target_coln][-10:])
plt.plot(dateval,val[target_coln])
plt.plot(datetest,test[target_coln])

dataoutput = data[target_coln].values.reshape(-1,1)
dataoutput = np.append(dataoutput, [[0]], axis=0)
scaleroutput.fit(dataoutput)
trainscaled = scalerinput.fit_transform(train)
testscaled = scalerinput.transform(test)
valscaled = scalerinput.transform(val)

x_train, y_train = create_sequences(trainscaled, seq_length, target_col)
x_test, y_test = create_sequences(testscaled, seq_length, target_col)
x_val, y_val = create_sequences(valscaled, seq_length, target_col)

TimeSteps=x_train.shape[1]
TotalFeatures=x_train.shape[2]
'''

DatetimeIndex(['2023-10-30', '2023-10-31', '2023-11-01', '2023-11-02',
               '2023-11-03'],
              dtype='datetime64[ns]', freq=None)
(463, 14)
(456, 3, 14)
(456, 5)


<ipython-input-32-e80945ba4489>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X), np.array(y), date, datepred, np.array(xpred)


IndexError: ignored

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 32, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=False))
regressor.add(Dense(units = 15,activation = 'relu'))
regressor.add(Dense(units = 15,activation = 'relu'))
#regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))
#regressor.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))
regressor.add(Dense(units = nextdays))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
regressor.compile(optimizer=opt, loss='mean_absolute_error',metrics=['mean_absolute_error'])
early_stop = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)

StartTime=time.time()
history = regressor.fit(x_train, y_train, epochs=500, batch_size=10, validation_data=(x_val,y_val), verbose=2)#,callbacks=[early_stop]
EndTime=time.time()

model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(TimeSteps, TotalFeatures)))

model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(32, activation='relu'))

model.add(Dense(nextdays))

opt = tf.keras.optimizers.Adam(learning_rate=0.005)

model.compile(optimizer=opt, loss='mean_squared_error',metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=500, batch_size=64,validation_data=(x_val,y_val),callbacks=[early_stop])#, validation_split=0.3
'''
print("## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ##')

stopped_epoch = early_stop.stopped_epoch
best_epoch = stopped_epoch - early_stop.patience if early_stop.stopped_epoch is not None else None
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['mean_absolute_error'], label='Training MAE')
plt.plot(history.history['val_mean_absolute_error'], label='Validation MAE')
plt.axvline(x=best_epoch, color='r', linestyle='--', label='Early Stopping Epoch')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('mean_absolute_error')
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True)) # Ensure integer values on x-axis
plt.legend()
plt.subplot(1, 2, 2)

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
#plt.axvline(x=best_epoch, color='r', linestyle='--', label='Early Stopping Epoch')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.legend()
plt.tight_layout()
plt.show()
'''

In [ ]:
predicted_Price = regressor.predict(x_train)
print(predicted_Price.shape)
print(predicted_Price)
predicted_Price= scaleroutput.inverse_transform(predicted_Price)
print(predicted_Price.shape)
print(predicted_Price)
plt.figure()
plt.plot(datetrain,predicted_Price[:,0],label='Pred')
plt.plot(datetrain,scaleroutput.inverse_transform(y_train)[:,0],label='Real')
plt.legend()
#plt.ylim(0, np.max(orig[2:trainsize]))
plt.grid()
plt.show()

predicted_Price = regressor.predict(x_val)
predicted_Price= scaleroutput.inverse_transform(predicted_Price)
print(predicted_Price.shape)
plt.figure()
plt.plot(dateval,predicted_Price[:,0],label='Pred')
plt.plot(dateval,scaleroutput.inverse_transform(y_val)[:,0],label='Real')
plt.legend()
#plt.ylim(0, np.max(orig[trainsize+2:-testsize]))
plt.grid()
plt.show()

predicted_Price = regressor.predict(x_test)
predicted_Price= scaleroutput.inverse_transform(predicted_Price)
print(predicted_Price.shape)
plt.figure()
plt.plot(datetest,predicted_Price[:,0],label='Pred1')
plt.plot(datetest[1:],predicted_Price[:-1,1],label='Pred2')
plt.plot(datetest[2:],predicted_Price[:-2,2],label='Pred3')
plt.plot(datetest,scaleroutput.inverse_transform(y_test)[:,-1],label='Real')
plt.legend()
#plt.ylim(0, np.max(orig[-testsize:]))
plt.grid()
plt.show()




predicted_Price = model.predict(x_train)
predicted_Price= scaleroutput.inverse_transform(predicted_Price)
print(predicted_Price.shape)
print(predicted_Price)
plt.figure()
plt.plot(datetrain,predicted_Price,label='Pred')
plt.plot(datetrain,scaleroutput.inverse_transform(y_train),label='Real')
plt.legend()
#plt.ylim(0, np.max(orig[2:trainsize]))
plt.grid()
plt.show()

predicted_Price = model.predict(x_val)
predicted_Price= scaleroutput.inverse_transform(predicted_Price)
print(predicted_Price.shape)
plt.figure()
plt.plot(dateval,predicted_Price,label='Pred')
plt.plot(dateval,scaleroutput.inverse_transform(y_val),label='Real')
plt.legend()
#plt.ylim(0, np.max(orig[trainsize+2:-testsize]))
plt.grid()
plt.show()

predicted_Price = model.predict(x_test)
predicted_Price= scaleroutput.inverse_transform(predicted_Price)
print(predicted_Price.shape)
plt.figure()
plt.plot(datetest,predicted_Price[:,-1],label='Pred')
plt.plot(datetest,scaleroutput.inverse_transform(y_test)[:,-1],label='Real')
plt.legend()
#plt.ylim(0, np.max(orig[-testsize:]))
plt.grid()
plt.show()


In [ ]:
predicted_Price = scaleroutput.inverse_transform(regressor.predict(x_train))
orig=train[target_coln]
plt.figure(figsize=(12, 4))
plt.plot(datetrain[-40:],predicted_Price[-40:])
plt.plot(datetrain[-40:],orig[-40:])
predicted_Price = scaleroutput.inverse_transform(regressor.predict(x_val))
orig=val[target_coln]
plt.plot(dateval[seq_length:],predicted_Price)
plt.plot(dateval[seq_length:],orig[seq_length:])
predicted_Price = scaleroutput.inverse_transform(regressor.predict(x_test))
orig=test[target_coln]
plt.plot(datetest[seq_length:],predicted_Price)
plt.plot(datetest[seq_length:],orig[seq_length:])
plt.legend(['Pred Train','Train','Pred Val','Val','Pred Test','Test'])
plt.figure(figsize=(12, 4))
plt.plot(datetest,predicted_Price)
plt.plot(datetest,orig)
plt.legend(['Pred Test','Test'])

In [ ]:
predicted_Price = scaleroutput.inverse_transform(regressor.predict(x_test))
orig=scaleroutput.inverse_transform(y_test)

print('Accuracy:', 100 - (100*(abs(orig-predicted_Price)/orig)).mean())

plt.plot(orig[0]*predicted_Price/max(predicted_Price), color = 'blue', label = 'Predicted Volume')
plt.plot(orig, color = 'lightblue', label = 'Original Volume')

plt.title('train Price Predictions')
plt.xlabel('Trading Date')
plt.ylabel('train Price')

plt.legend()
fig=plt.gcf()
fig.set_figwidth(20)
fig.set_figheight(6)
plt.show()

plt.figure(figsize=(12, 4))
plt.plot(predicted_Price, color = 'blue', label = 'Predicted Volume')
plt.plot(orig, color = 'lightblue', label = 'Original Volume')

plt.title('train Price Predictions')
plt.xlabel('Trading Date')
plt.ylabel('train Price')

plt.legend()
fig=plt.gcf()
fig.set_figwidth(20)
fig.set_figheight(6)
plt.show()

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(
      LSTM(
          # Tune number of units separately.
          units=hp.Int("units", min_value=32, max_value=512, step=32),
          activation=hp.Choice("activation", ["relu", "tanh"]),
          return_sequences=True
      )
  )
  model.add(
      LSTM(
          # Tune number of units separately.
          units=hp.Int("units1", min_value=32, max_value=64, step=2),
          activation=hp.Choice("activation1", ["relu", "tanh"]),
          return_sequences=False
          )
      )
  if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
  model.add(layers.Dense(1))
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
      loss="mse",
      metrics=["accuracy"])
  return model

build_model(keras_tuner.HyperParameters())

hp = keras_tuner.HyperParameters()

tuner = keras_tuner.RandomSearch(
  build_model,
  hyperparameters=hp,
  objective=keras_tuner.Objective("accuracy", direction="max"),
  overwrite=True,
  max_trials=10,
  max_retries_per_trial=3)

early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

tuner.search(x_train, y_train, epochs=50, batch_size=16,validation_data=(x_val, y_val))
tuner.search_space_summary()
tuner.results_summary(1)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(tuner.get_best_hyperparameters()[0])
best_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=["accuracy"])
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
best_model.fit(x_train, y_train, epochs=100, batch_size=16,validation_data=(x_val, y_val),callbacks=[early_stop])#, validation_split=0.3
best_model.summary()
keras.utils.plot_model(best_model,show_shapes=True)

predicted_Price = scaleroutput.inverse_transform(best_model.predict(x_test))
orig=scaleroutput.inverse_transform(y_test)

# Accuracy of the predictions
print('Accuracy:', 100 - (100*(abs(orig-predicted_Price)/orig)).mean())


plt.plot(orig[0]*predicted_Price/max(predicted_Price), color = 'blue', label = 'Predicted Volume')
plt.plot(orig, color = 'lightblue', label = 'Original Volume')

plt.title('train Price Predictions')
plt.xlabel('Trading Date')
plt.ylabel('train Price')

plt.legend()
fig=plt.gcf()
fig.set_figwidth(20)
fig.set_figheight(6)
plt.show()

plt.figure(figsize=(12, 4))
plt.plot(predicted_Price, color = 'blue', label = 'Predicted Volume')
plt.plot(orig, color = 'lightblue', label = 'Original Volume')

plt.title('train Price Predictions')
plt.xlabel('Trading Date')
plt.ylabel('train Price')

plt.legend()
fig=plt.gcf()
fig.set_figwidth(20)
fig.set_figheight(6)
plt.show()

In [ ]:
a = build_model(tuner.get_best_hyperparameters()[0])
a.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"])
a.fit(x_train, y_train, epochs=1, batch_size=64)
a.summary()
keras.utils.plot_model(a,show_shapes=True)

In [ ]:
data = pd.read_csv('data.csv')
seq_length = 2
target = 'Close'
dayspan = 90
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)
train = data
scaler = MinMaxScaler()

test = train.tail(dayspan)
train = train.iloc[:-dayspan]
train.reset_index(drop=True, inplace=True)

target_col = target
features = train.columns
X = train[features].values
y = train[target_col].values
dx = test[features].values
dy = test[target_col].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
scaler = MinMaxScaler()
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
dx = scaler.transform(dx)
y_train = scaler1.fit_transform(y_train.reshape(-1,1))
y_test = scaler1.transform(y_test.reshape(-1,1))

X_train_seq, temp = create_sequences(X_train[:-1], seq_length)
y_train_seq, temp = create_sequences(y_train, seq_length)
X_test_seq, temp = create_sequences(X_test[:-1], seq_length)
y_test_seq, temp = create_sequences(y_test, seq_length)
dxx, dyy = create_sequences(dx, seq_length)

model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(seq_length, len(features))))

model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(32, activation='relu'))

model.add(Dense(1))

opt = tf.keras.optimizers.Adam(learning_rate=0.005)

model.compile(optimizer=opt, loss='mean_squared_error',metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)
history = model.fit(X_train_seq, y_train[3:], epochs=500, batch_size=64,validation_data=(X_test_seq, y_test[3:]),callbacks=[early_stop])#, validation_split=0.3
print(X_train_seq.shape,y_train_seq.shape,y_train.shape)

scale=MinMaxScaler()
scale.min_,scale.scale_=scaler.min_[0],scaler.scale_[0]

predbest = scale.inverse_transform(model.predict(dxx))
predbest = predbest.reshape(-1,1)
real_prices = dy[:-seq_length]

days = list(range(1, dayspan-seq_length+1))

plt.figure(figsize=(12, 6))
plt.plot(days, real_prices[0]*predbest/max(predbest), label="Predicted Prices", marker='o', linestyle='-', color='b')
#plt.plot(days, predbest, label="Predicted normal Prices", marker='o', linestyle='-', color='b')
plt.plot(days, real_prices, label="Real Prices", marker='x', linestyle='-', color='g')
plt.xlabel("Days")
plt.ylabel("train Price")
plt.title("Predicted vs. Real train Prices")
plt.legend()
plt.grid(True)

plt.show()

plt.figure(figsize=(12, 6))
plt.plot(days, predbest, label="Predicted normal Prices", marker='o', linestyle='-', color='b')
plt.plot(days, real_prices, label="Real Prices", marker='x', linestyle='-', color='g')
plt.xlabel("Days")
plt.ylabel("train Price")
plt.title("Predicted vs. Real train Prices")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
!pip install yfinance --upgrade --no-cache-dir
from pandas_datareader import data as pdr
from collections import deque
import random
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
f= predbest.copy()
name = 'Q-learning agent'
class Agent:
    def __init__(self, state_size, window_size, trend, skip, batch_size):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.action_size = 3
        self.batch_size = batch_size
        self.memory = deque(maxlen = 1000)
        self.inventory = []
        self.gamma = 0.95
        self.epsilon = 0.5
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()
        self.X = tf.placeholder(tf.float32, [None, self.state_size])
        self.Y = tf.placeholder(tf.float32, [None, self.action_size])
        feed = tf.layers.dense(self.X, 256, activation = tf.nn.relu)
        self.logits = tf.layers.dense(feed, self.action_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(
            self.cost
        )
        self.sess.run(tf.global_variables_initializer())

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(
            self.sess.run(self.logits, feed_dict = {self.X: state})[0]
        )

    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 2):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def replay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size, l):
            mini_batch.append(self.memory[i])
        replay_size = len(mini_batch)
        X = np.empty((replay_size, self.state_size))
        Y = np.empty((replay_size, self.action_size))
        states = np.array([a[0][0] for a in mini_batch])
        new_states = np.array([a[3][0] for a in mini_batch])
        Q = self.sess.run(self.logits, feed_dict = {self.X: states})
        Q_new = self.sess.run(self.logits, feed_dict = {self.X: new_states})
        for i in range(len(mini_batch)):
            state, action, reward, next_state, done = mini_batch[i]
            target = Q[i]
            target[action] = reward
            if not done:
                target[action] += self.gamma * np.amax(Q_new[i])
            X[i] = state
            Y[i] = target
        cost, _ = self.sess.run(
            [self.cost, self.optimizer], feed_dict = {self.X: X, self.Y: Y}
        )
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return cost

    def buy(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        for t in range(0, len(self.trend) - 1, self.skip):
            action = self.act(state)
            next_state = self.get_state(t + 1)
            if action == 1 and initial_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
                print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.trend[t], initial_money))
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((close[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
                    % (t, close[t], invest, initial_money)
                )
            state = next_state
        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, invest

    def train(self, iterations, checkpoint, initial_money):
      for i in range(iterations):
          total_profit = 0
          inventory = []
          state = self.get_state(0)
          starting_money = initial_money
          cost = 0  # Initialize 'cost' outside the loop
          for t in range(0, len(self.trend) - 1, self.skip):
              action = self.act(state)
              next_state = self.get_state(t + 1)
              if action == 1 and starting_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                  inventory.append(self.trend[t])
                  starting_money -= self.trend[t]
              elif action == 2 and len(inventory) > 0:
                  bought_price = inventory.pop(0)
                  total_profit += self.trend[t] - bought_price
                  starting_money += self.trend[t]
              invest = ((starting_money - initial_money) / initial_money)
              self.memory.append((state, action, invest, next_state, starting_money < initial_money))
              state = next_state
              batch_size = min(self.batch_size, len(self.memory))
              cost = self.replay(batch_size)
          if (i+1) % checkpoint == 0:
              print(f'epoch: {i + 1}, total rewards: {total_profit:.3f}, cost: {cost}, total money: {starting_money}')



In [ ]:
close = orig.T
initial_money = 10000
window_size = 90
skip = 2
batch_size = 32
agent = Agent(state_size = window_size,
              window_size = window_size,
              trend = close,
              skip = skip,
              batch_size = batch_size)
agent.train(iterations = 200, checkpoint = 10, initial_money = initial_money)

In [ ]:
states_buy, states_sell, total_gains, invest = agent.buy(initial_money = initial_money)

In [ ]:
fig = plt.figure(figsize = (15,5))
plt.plot(close, color='r', lw=2.)
plt.plot(close, '^', markersize=10, color='m', label = 'buying signal', markevery = states_buy)
plt.plot(close, 'v', markersize=10, color='k', label = 'selling signal', markevery = states_sell)
plt.title('total gains %f, total investment %f%%'%(total_gains, invest))
#plt.legend()
plt.savefig(name+'.png')
plt.show()